## Generate centroids of multipolygons

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import geopandas as gpd
import couchdb
import os
import json

In [10]:
# load boundaries of states and SUAs
state_gdf = gpd.read_file("../data/Digital boundaries/state/STE_2016_AUST.shp")
sua_gdf = gpd.read_file("../data/Digital boundaries/SUA/SUA_2016_AUST.shp")

In [11]:
def centroid_state(gdf, name):
    gdf['centroids'] = gdf.centroid
    gdf['lat'] = gdf['centroids'].y
    gdf['lng'] = gdf['centroids'].x
    gdf['name'] = gdf[name].str.lower()
    gdf = gdf[['name', 'lat', 'lng']]
    return gdf

In [12]:
def centroid_sua(gdf, name):
    gdf['centroids'] = gdf.centroid
    gdf['lat'] = gdf['centroids'].y
    gdf['lng'] = gdf['centroids'].x
    gdf['name'] = gdf[name]
    gdf = gdf[['name', 'lat', 'lng']]
    return gdf

In [13]:
state_gdf_finalized = centroid_state(state_gdf, 'STE_NAME16')
sua_gdf_finalized = centroid_sua(sua_gdf, 'SUA_NAME16')

In [14]:
geo_center = pd.concat([state_gdf_finalized, sua_gdf_finalized])
geo_center.head(9)

,name,lat,lng
0,new south wales,-32.165600,147.008154
1,victoria,-36.850094,144.304288
2,queensland,-22.567052,144.545851
3,south australia,-30.108162,135.831738
4,western australia,-25.466471,122.181775
5,tasmania,-41.936457,146.595152
6,northern territory,-19.414739,133.364913
7,australian capital territory,-35.488766,149.002523
8,other territories,-20.797265,128.368543


In [15]:
geo_center['name'][8] = 'offshore territories'
geo_center.head(9)

,name,lat,lng
0,new south wales,-32.165600,147.008154
1,victoria,-36.850094,144.304288
2,queensland,-22.567052,144.545851
3,south australia,-30.108162,135.831738
4,western australia,-25.466471,122.181775
5,tasmania,-41.936457,146.595152
6,northern territory,-19.414739,133.364913
7,australian capital territory,-35.488766,149.002523
8,offshore territories,-20.797265,128.368543


In [16]:
geo_center.shape

(119, 3)

In [17]:
with open('../data//Digital boundaries/geo_center.json', 'w') as f:
    geo_center.to_json(f, orient='records')

In [30]:
user = 'admin'
psw = 'qazwsxedc'
node_ip = '172.26.130.9'
# connect to the CouchDB server
server = couchdb.Server(f'http://{user}:{psw}@{node_ip}:5984')

# select the database
db = server['geocenter']

i = 0
package = []

with open(r"../data//Digital boundaries/geo_center.json", "r") as f:
    data = json.load(f)
    for item in data:
        db.save(item)